# Autocompletion Using MobileBERT

In [2]:
#install transformers if you haven't already:
## TODO


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Import transofmers and import model
from transformers import pipeline
model_name = 'google/mobilebert-uncased'
word_completion = pipeline('text-generation', model=model_name, tokenizer=model_name)

In [ ]:
# TESTING

# Set the word prefix
prefix = 'compu'

# Generate a completion for the word prefix
completion = word_completion(prefix, max_length=10, num_return_sequences=1) #can change this to however many suggestions we want


# Print the completion
text = completion[0]['generated_text'].strip()
print('Completion:', text)


